In [1]:
pip install benchmarkfcns==2.5.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.0/148.0 kB 2.7 MB/s eta 0:00:00


In [2]:
import torch
import matplotlib.pyplot as plt
from benchmarkfcns import schwefel223

Schwefel 2.23 Function:
$$f(\mathbf{x})=f(x_1, ..., x_n)=\sum_{i=1}^{n}x_i^{10}
$$

Gradient Calculation:
$$\frac{df}{dx_i} = \frac{d}{dx_i}(x_i^{10}) = 10x_i^9$$
$$∇f(x) = (10x_1^9, 10x_2^9, …, 10x_n^9)$$

Hessian Calculation:
$$H(\mathbf{x}) = \text{diag}\left(90x_1^8, 90x_2^8, \dots, 90x_n^8\right)
$$

In [3]:
n = 5

#function
def f(x):
  result = torch.sum((x ** 10))
  return result

#gradient
def grad_f(x):
  result = 10 * (x ** 9)
  return result

#hessian
def hes(x):
  dim = x.shape[0]
  diag_elements = 90 * x ** 8
  H = torch.diag(diag_elements.view(-1))
  return H

#gradient using AD
def grad_f_AD(x):
  dim = x.shape[0]
  assert x.shape == (dim, 1) # ensuring input is valid in context
  #track the input
  x.requires_grad = True
  y = f(x)
  dy = torch.autograd.grad(outputs=y, inputs=x,grad_outputs=torch.ones_like(y), create_graph=True)[0]

  #detach variables to release memeory
  x = x.detach()
  y = y.detach()
  dy = dy.detach()

  return dy

#hessian using AD
def hes_f_AD(x):
  x.requires_grad = True
  y = f(x)
  dy = torch.autograd.grad(outputs = y, inputs = x, grad_outputs = torch.ones_like(y), create_graph = True)[0]

  #initialize hessian
  dim = x.shape[0]
  H = torch.zeros((dim, dim), dtype=x.dtype, device=x.device)

  #identity matrix
  I = torch.eye(dim, dtype=x.dtype, device = x.device)

  # for loop through identity matrix to compute second derivates
  for i in range(dim):
    grad_output = I[i].unsqueeze(-1)
    second_grad = torch.autograd.grad(outputs=dy, inputs = x, grad_outputs=grad_output, create_graph=True)[0]
    H[i] = second_grad.squeeze()

  #detach variables
  H = H.detach()
  dy = dy.detach()
  x = x.detach()

  return H

In [ ]:
# test function
X = torch.rand(1000, n)
f1Out = schwefel223(X)
passed = True

for i in range(1000):
  f2 = f(X[i])
  # print(X[i].shape)
  if abs(f1Out[i] - f2) > .0005:
    passed = False
    print('error')
    print(f1Out[i])
    #print(X[i])
    print(f2)
    break
if passed:
  print('passed')

passed


In [11]:
dim = n
x = torch.randn(dim,1)
v = torch.randn_like(x)
v = v / torch.norm(v)

max_iters = 32
h = torch.zeros(max_iters)
err0 = torch.zeros(max_iters) # error from zero order T.P.
err1 = torch.zeros(max_iters) # error from first order T.P.
err2 = torch.zeros(max_iters) # error from second order T.P.
for i in range(max_iters):
  h[i] = 2**(-i) # iterate h down

  #regular
  # fv = f(x + h[i]*v)
  # T0 = f(x)
  # T1 = T0 + h[i] * (grad_f(x)).T @ v
  # T2 = T1 + .5*(h[i]**2) * v.T @ hes(x) @ v

  #using AD
  fv = f(x+h[i]*v).detach()
  T0 = f(x).detach()
  T1 = T0 + h[i] * (grad_f_AD(x)).T @ v
  T2 = T1 + .5*(h[i]**2) * v.T @ hes_f_AD(x) @ v

  err0[i] = torch.norm(fv - T0)
  err1[i] = torch.norm(fv - T1)
  err2[i] = torch.norm(fv - T2)
  print('h: %.3e, \t err0: %.3e, \t err1: %.3e, \t err2: %.3e' % (h[i], err0[i], err1[i], err2[i]))

h: 1.000e+00, 	 err0: 2.424e-01, 	 err1: 1.071e+00, 	 err2: 2.569e+00
h: 5.000e-01, 	 err0: 7.216e-02, 	 err1: 3.419e-01, 	 err2: 5.679e-01
h: 2.500e-01, 	 err0: 7.852e-02, 	 err1: 1.285e-01, 	 err2: 9.893e-02
h: 1.250e-01, 	 err0: 6.160e-02, 	 err1: 4.192e-02, 	 err2: 1.494e-02
h: 6.250e-02, 	 err0: 3.961e-02, 	 err1: 1.215e-02, 	 err2: 2.066e-03
h: 3.125e-02, 	 err0: 2.260e-02, 	 err1: 3.282e-03, 	 err2: 2.720e-04
h: 1.562e-02, 	 err0: 1.209e-02, 	 err1: 8.535e-04, 	 err2: 3.490e-05
h: 7.812e-03, 	 err0: 6.252e-03, 	 err1: 2.177e-04, 	 err2: 4.411e-06
h: 3.906e-03, 	 err0: 3.180e-03, 	 err1: 5.495e-05, 	 err2: 5.811e-07
h: 1.953e-03, 	 err0: 1.604e-03, 	 err1: 1.384e-05, 	 err2: 4.470e-08
h: 9.766e-04, 	 err0: 8.053e-04, 	 err1: 3.435e-06, 	 err2: 3.725e-08
h: 4.883e-04, 	 err0: 4.035e-04, 	 err1: 8.494e-07, 	 err2: 1.490e-08
h: 2.441e-04, 	 err0: 2.019e-04, 	 err1: 2.384e-07, 	 err2: 2.235e-08
h: 1.221e-04, 	 err0: 1.011e-04, 	 err1: 3.725e-08, 	 err2: 1.490e-08
h: 6.104e-05, 	 err0

In [ ]:
plt.loglog(h, err0, linewidth = 3)
plt.loglog(h, err1, linewidth = 3)
plt.loglog(h, err2, linewidth = 3)
plt.legend(['$\|f(x+hv) - T_0(x)\|$', '$\|f(x+hv) - T_1(x)\|$', '$\|f(x+hv) - T_2(x)\|$'])
plt.xticks(fontsize =15)
plt.yticks(fontsize =15)

(array([1.e-10, 1.e-09, 1.e-08, 1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03,
        1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
 [Text(0, 1e-10, '$\\mathdefault{10^{-10}}$'),
  Text(0, 1e-09, '$\\mathdefault{10^{-9}}$'),
  Text(0, 1e-08, '$\\mathdefault{10^{-8}}$'),
  Text(0, 1e-07, '$\\mathdefault{10^{-7}}$'),
  Text(0, 1e-06, '$\\mathdefault{10^{-6}}$'),
  Text(0, 1e-05, '$\\mathdefault{10^{-5}}$'),
  Text(0, 0.0001, '$\\mathdefault{10^{-4}}$'),
  Text(0, 0.001, '$\\mathdefault{10^{-3}}$'),
  Text(0, 0.01, '$\\mathdefault{10^{-2}}$'),
  Text(0, 0.1, '$\\mathdefault{10^{-1}}$'),
  Text(0, 1.0, '$\\mathdefault{10^{0}}$'),
  Text(0, 10.0, '$\\mathdefault{10^{1}}$'),
  Text(0, 100.0, '$\\mathdefault{10^{2}}$')])